In [350]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0
import folium

### Information below contains a chart with consolidated Neighborhood based on Borough and Zip code. Data was obtained from wikipedia. At the following link https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.


In [351]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # download data
df = df[0]# Select appropriate index for data
df = df[df['Borough']!='Not assigned'] # Remove all boroughs that don't have an assigned zip code
df['Borough'].unique() # checking no other data anomilies exist
df = df.groupby(['Postcode','Borough'], as_index = False, sort = False).agg(','.join) # Comebine neighborhoods within the same zip code and Borough
df['Neighbourhood'] = np.where(df["Neighbourhood"] == 'Not assigned', df['Borough'], df['Neighbourhood']) # Missing values in Neighborhood replaced with Borough.
df.shape

(103, 3)

### Below we obtain a separate file which provides longitude and latitude of specific zip codes, we will merge the existing table above with the new information


In [352]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [353]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df = pd.merge(df,df2,how = 'left', left_on=df['Postcode'], right_on=df2['Postal Code'])
df = df[['Postcode','Borough', 'Neighbourhood', 'Latitude','Longitude']]
df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Create a map of Toronto

In [354]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_Location")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [355]:
# create map of Toronto using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

### There seems to be a high density of points located at Downtown Toronto, I will be concentrating on that are for further exploration.

In [356]:
df_toronto = df[df['Borough'] == 'Downtown Toronto']

In [357]:
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [358]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronro are 43.6541737, -79.3808116451341.


In [359]:
# create map of Toronto using latitude and longitude values
Dtown_Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Dtown_Toronto_map)  
    
Dtown_Toronto_map

### Define Foursquare Credentials

In [360]:
CLIENT_ID = 'BWU4F1HVP4WGYXL235HO1YKGGM1ZH1CC130AVLPHR0DRHH52' # your Foursquare ID
CLIENT_SECRET = 'ZDLJGFVIQQ2OPDD5DN1VKPM30PDBPY0S0HLJRFGEVMVQZQ54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


### Obtain venues top 20 venues within 100 meters


In [361]:
LIMIT = 20 #limit for venues defined
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [362]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


### Resulting data frame

In [363]:
toronto_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,"Harbourfront,Regent Park",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Ryerson,Garden District",43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery
3,"Ryerson,Garden District",43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Coffee Shop
4,St. James Town,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant


### Grouping of each category within a neighborhood and standardizing the frequency of each occurancy by average

In [364]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(3)

,Neighbourhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,...,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.083333,0.083333,0.083333,0.0,0.0,0.083333
1,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [365]:
toronto_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,...,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Ryerson,Garden District",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Ryerson,Garden District",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,St. James Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Most common venue 20 venues

In [366]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Adelaide,King,Richmond",Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,Greek Restaurant,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
1,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
2,"Cabbagetown,St. James Town",Italian Restaurant,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,Cupcake Shop,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
3,Central Bay Street,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
4,"Chinatown,Grange Park,Kensington Market",Cocktail Bar,Café,Farmers Market,Liquor Store,Thrift / Vintage Store,Bar,Concert Hall,Garden,Food Court,...,Art Gallery,Deli / Bodega,Cupcake Shop,Bakery,College Gym,Bookstore,Coffee Shop,Gastropub,Bank,Burrito Place


### Cluster Neighborhoods N=7

In [367]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged = toronto_merged.dropna() # drop any neighborhoods that didnt have any data

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Japanese Restaurant,Performing Arts Venue,Hostel,Italian Restaurant,...,Concert Hall,College Gym,Cocktail Bar,Garden,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,5.0,Nightclub,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


### Create a cluster map

In [368]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster examination

In [369]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Looks like a very healthy neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,"Harbourfront,Regent Park",0.0,Spa,Breakfast Spot,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Bookstore,Bar
9,"Ryerson,Garden District",0.0,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
15,St. James Town,0.0,Japanese Restaurant,Performing Arts Venue,Hostel,Italian Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Cocktail Bar,Garden,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore
24,Central Bay Street,0.0,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,...,Concert Hall,College Gym,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
30,"Adelaide,King,Richmond",0.0,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Coffee Shop,Concert Hall,Food Court,Garden,Hotel,...,Sushi Restaurant,Tea Room,Burrito Place,Deli / Bodega,Cupcake Shop,College Gym,Cocktail Bar,Café,Sandwich Place,Farmers Market
42,"Design Exchange,Toronto Dominion Centre",0.0,Coffee Shop,Restaurant,Gym,Deli / Bodega,Vegetarian / Vegan Restaurant,Cocktail Bar,Food Court,Fast Food Restaurant,...,Concert Hall,College Gym,Café,Gastropub,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
48,"Commerce Court,Victoria Hotel",0.0,American Restaurant,Salad Place,Deli / Bodega,Gym,Coffee Shop,Café,Burrito Place,Pub,...,Sandwich Place,Soup Place,Art Gallery,Bakery,Seafood Restaurant,Sushi Restaurant,College Gym,Farmers Market,Cupcake Shop,Concert Hall
84,"Chinatown,Grange Park,Kensington Market",0.0,Cocktail Bar,Café,Farmers Market,Liquor Store,Thrift / Vintage Store,Bar,Concert Hall,Garden,...,Art Gallery,Deli / Bodega,Cupcake Shop,Bakery,College Gym,Bookstore,Coffee Shop,Gastropub,Bank,Burrito Place
92,Stn A PO Boxes 25 The Esplanade,0.0,Pub,Steakhouse,Bar,Breakfast Spot,Hotel,Italian Restaurant,Concert Hall,Cocktail Bar,...,Deli / Bodega,Cupcake Shop,College Gym,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Garden,Burrito Place,Burger Joint,Building
97,"First Canadian Place,Underground city",0.0,Café,Building,Gluten-free Restaurant,Gym / Fitness Center,Deli / Bodega,Cupcake Shop,Coffee Shop,Burger Joint,...,Bank,Thai Restaurant,Breakfast Spot,Bookstore,Burrito Place,Garden,Cocktail Bar,Bar,College Gym,Concert Hall


In [370]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Nothing too special about this neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
36,"Harbourfront East,Toronto Islands,Union Station",1.0,Gym,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [371]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Art/Performing Hall centered neighborhood

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
91,Rosedale,2.0,Park,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [372]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#THis neighborhood probably has the most active night life

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
87,"CN Tower,Bathurst Quay,Island airport,Harbourf...",3.0,Performing Arts Venue,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [373]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Food concentrated area

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
80,"Harbord,University of Toronto",4.0,College Gym,Vegetarian / Vegan Restaurant,Cocktail Bar,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,Coffee Shop,Café,Gluten-free Restaurant,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar,Bank


In [374]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#College town

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
25,Christie,5.0,Nightclub,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar


In [375]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
#Recreational area

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
96,"Cabbagetown,St. James Town",6.0,Italian Restaurant,Vegetarian / Vegan Restaurant,Gluten-free Restaurant,Garden,Food Court,Fast Food Restaurant,Farmers Market,Deli / Bodega,...,College Gym,Coffee Shop,Cocktail Bar,Café,Burrito Place,Burger Joint,Building,Breakfast Spot,Bookstore,Bar
